In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# from lightgbm import LGBMRegressor
import lightgbm as lgb
import gc
from sklearn.model_selection import train_test_split
from sklearn.metrics import median_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
import matplotlib.pyplot as plt
# plt.style.use('ggplot') #グラフのスタイル
# plt.rcParams['figure.figsize'] = [12, 9] # グラフサイズ設定

# Read Data

In [ ]:
df = pd.read_parquet("/kaggle/input/2024ucs654labeval1007/Lab Eval/train_data.parquet")
test = pd.read_parquet("/kaggle/input/2024ucs654labeval1007/Lab Eval/test_data.parquet")

In [ ]:
df.head(5)

In [ ]:
df.tail(5)

In [ ]:
df.columns.unique()

In [ ]:
df.dtypes

In [ ]:
test.head(5)

In [ ]:
test.tail(5)

In [ ]:
test.columns.unique()

In [ ]:
print(set(df.columns)^set(test.columns))

# Feature Engineering

In [ ]:
target = 'T7'

In [ ]:
features = df.columns[2:-10]
features

In [ ]:
features5 = list(np.array_split(features, 5))
features5
features10 = list(np.array_split(features, 10))
features10

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(df[features], df[target],test_size=0.4)

## PCA

In [ ]:
PCAs = {}
n = 8
X_train_new = pd.DataFrame()
X_valid_new = pd.DataFrame()
X_test_new = pd.DataFrame()
for i,feature in enumerate(features5):
    pca = PCA(n_components=n)
    columns = [f'feature5_{i}_{j+1}' for j in range(n)]
    tmp_train = pd.DataFrame(pca.fit_transform(X_train[feature]), columns=columns)
    tmp_valid = pd.DataFrame(pca.transform(X_valid[feature]), columns=columns)
    tmp_test = pd.DataFrame(pca.transform(test[feature]), columns=columns)
    X_train_new = pd.concat([X_train_new,tmp_train], axis=1)
    X_valid_new = pd.concat([X_valid_new,tmp_valid], axis=1)
    X_test_new = pd.concat([X_test_new,tmp_test], axis=1)
    PCAs[f'feature5_{i}'] = pca

for i,feature in enumerate(features10):
    pca = PCA(n_components=n)
    columns = [f'feature10_{i}_{j+1}' for j in range(n)]
    tmp_train = pd.DataFrame(pca.fit_transform(X_train[feature]), columns=columns)
    tmp_valid = pd.DataFrame(pca.transform(X_valid[feature]), columns=columns)
    tmp_test = pd.DataFrame(pca.transform(test[feature]), columns=columns)
    X_train_new = pd.concat([X_train_new,tmp_train], axis=1)
    X_valid_new = pd.concat([X_valid_new,tmp_valid], axis=1)
    X_test_new = pd.concat([X_test_new,tmp_test], axis=1)
    PCAs[f'feature10_{i}'] = pca


In [ ]:
X_train_new

In [ ]:
X_valid_new

In [ ]:
X_test_new

## LightGBM

In [ ]:
params = {
    'n_estimators': 500,
    'boosting_type': 'gbdt',
    'learning_rate': 0.05,
    'metric': 'rmse',
    'colsample_bytree': 0.8,
    'seed': 0,
}

In [ ]:
model = lgb.LGBMRegressor(**params)
model.fit(
    X_train_new, y_train,
    eval_set=[(X_valid_new, y_valid)],
    callbacks=[
        lgb.early_stopping(stopping_rounds=50, verbose=True),
        lgb.log_evaluation(100),
    ],
)
print(model.best_score_["valid_0"]["rmse"])

In [ ]:
model.predict(X_test_new)

# Submission

In [ ]:
submission = pd.DataFrame({'ID':test.index.values,'T7': np.zeros(len(test))})
submission['T7'] = model.predict(X_test_new)
submission.to_csv("submission.csv", index=False)

In [ ]:
submission

# Advance

In [ ]:
R2 = round(model.score(X_train_new, y_train),2)
# グラフ
plt.scatter(y_train, model.predict(X_train_new))
plt.title('Train data  -  R2='+str(R2))
plt.xlabel("actual")
plt.ylabel("predicted")
plt.show()

## Practical 1

In [ ]:
"""
# pipeline
PCR_model = make_pipeline(StandardScaler(), 
                          PCA(), 
                          LinearRegression())
# range
param = {
    'pca__n_components' : np.arange(1, len(X[0])+1)
}
# gridsearch
PCR_grid = GridSearchCV(PCR_model,
                        param_grid=param,
                        cv=10
                        ) 
PCR_grid.fit(X_train, y_train)
# best param
PCR_grid.best_params_
"""

## Practical 2

In [ ]:
"""
cv = KFold(n_splits=2)
for fold, (trn_idx, val_idx) in enumerate(cv.split(X), start=0):
    trn_x = X[trn_idx, :]
    trn_y = y[trn_idx]
    val_x = X[val_idx, :]
    val_y = y[val_idx]
    
    model = lgb.LGBMRegressor(**params)
    model.fit(
        trn_x, trn_y,
        eval_set=[(val_x, val_y)],
        callbacks=[
            lgb.early_stopping(stopping_rounds=5, verbose=True),
            lgb.log_evaluation(100),
        ],
    )
    print(model.best_score_["valid_0"]["rmse"])
"""